# Simple ML Jump (STUDENT VERSION)
Billy Hau - June 29, 2022

- Data Cleaning / Feature Engineering / Modeling 
- Deployment

In [222]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

In [223]:
# Read CSV File
df = pd.read_csv("./data.csv")
df


,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,21.340134,15.113852,-1.000000,-1.000000,False,True,True,False
1,42.000000,15.413610,57.000000,14.252161,False,False,False,False
2,38.155075,15.413610,53.444794,14.252161,False,True,False,False
3,34.056351,15.413610,49.654922,14.252161,False,True,False,False
4,30.188143,15.413610,46.078190,14.252161,False,True,False,False
...,...,...,...,...,...,...,...,...
24526,7.784975,15.327479,19.591297,15.466596,False,True,False,False
24527,3.940220,15.327479,15.711645,15.466596,False,True,False,False
24528,0.101849,15.327479,11.838436,15.466596,False,False,True,False
24529,39.339436,15.992887,54.632328,14.232322,False,True,False,False


In [224]:
# What do each of the columns mean?

In [225]:
df['Index'] = df.index
print(df['Index'])
jump = df[df['Jump'] == True]

dead = jump['Dead'].to_numpy()
# dead.value_counts()
for i in range(dead.shape[0]):
    #print(jump.index[i])
    try:
        if (dead[i + 1] == True):
            print(df['Index'])
            print(df['Index'] == jump.index[i])
            print(df[df.index == jump.index[i]].index)
            df.drop(df[df.index == jump.index[i]].index, inplace=True)
    except:
        pass
df

0            0
1            1
2            2
3            3
4            4
         ...  
24526    24526
24527    24527
24528    24528
24529    24529
24530    24530
Name: Index, Length: 24531, dtype: int64
0            0
1            1
2            2
3            3
4            4
         ...  
24526    24526
24527    24527
24528    24528
24529    24529
24530    24530
Name: Index, Length: 24531, dtype: int64
0        False
1        False
2        False
3        False
4        False
         ...  
24526    False
24527    False
24528    False
24529    False
24530    False
Name: Index, Length: 24531, dtype: bool
Int64Index([1978], dtype='int64')
0            0
1            1
2            2
3            3
4            4
         ...  
24526    24526
24527    24527
24528    24528
24529    24529
24530    24530
Name: Index, Length: 24530, dtype: int64
0        False
1        False
2        False
3        False
4        False
         ...  
24526    False
24527    False
24528    False
24529   

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored,Index
0,21.340134,15.113852,-1.000000,-1.000000,False,True,True,False,0
1,42.000000,15.413610,57.000000,14.252161,False,False,False,False,1
2,38.155075,15.413610,53.444794,14.252161,False,True,False,False,2
3,34.056351,15.413610,49.654922,14.252161,False,True,False,False,3
4,30.188143,15.413610,46.078190,14.252161,False,True,False,False,4
...,...,...,...,...,...,...,...,...,...
24526,7.784975,15.327479,19.591297,15.466596,False,True,False,False,24526
24527,3.940220,15.327479,15.711645,15.466596,False,True,False,False,24527
24528,0.101849,15.327479,11.838436,15.466596,False,False,True,False,24528
24529,39.339436,15.992887,54.632328,14.232322,False,True,False,False,24529


In [226]:
# Are there data we don't care about?
df.drop(df[df['Dead'] == True].index, inplace=True)
df = df.drop(["Grounded", "Dead", "Scored"], inplace=False, axis=1)
df

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Index
1,42.000000,15.413610,57.000000,14.252161,False,1
2,38.155075,15.413610,53.444794,14.252161,False,2
3,34.056351,15.413610,49.654922,14.252161,False,3
4,30.188143,15.413610,46.078190,14.252161,False,4
5,26.079243,15.413610,42.278908,14.252161,False,5
...,...,...,...,...,...,...
24525,11.858165,15.327479,23.701456,15.466596,False,24525
24526,7.784975,15.327479,19.591297,15.466596,False,24526
24527,3.940220,15.327479,15.711645,15.466596,False,24527
24529,39.339436,15.992887,54.632328,14.232322,False,24529


In [227]:
jumpCount = df[df['Jump'] == True].shape[0]
jumpCount

474

In [228]:
noJumpCount = df[df['Jump'] == False].shape[0]
noJumpCount

23154

In [229]:
df_toDrop = df[df['Jump'] == False].sample(noJumpCount - jumpCount)
df.drop(df_toDrop.index, inplace=True)

In [230]:
# Create the Input DataFrame
X = df[['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance', 'Bar 2 Speed']]
X = X.to_numpy()

In [231]:
# Create the Output DataFrame and Map True / False to 1 / 0
Y = df[['Jump']]

def bool2Num(row):
    if row['Jump']:
        return 1
    else:
        return 0

Y['Jump'] = Y.apply(bool2Num, axis='columns')
Y = Y['Jump'].to_numpy()

/tmp/ipykernel_1218930/808815421.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Jump'] = Y.apply(bool2Num, axis='columns')


In [232]:
# Train Test Split with test size set to 20%
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [233]:
from sklearn.linear_model import LogisticRegression

# Define and Build Model (Logistic Regression)
model = RandomForestClassifier()
model.fit(x_train, y_train)


RandomForestClassifier()

In [234]:
predict = model.predict(x_test)

In [235]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [236]:
# Print Out Prediction Accuraccy
accuracy_score(y_test, predict)

0.8881789137380192

In [237]:
# Try a Different Model (Random Forest)
"""model2 = ________________________________
model2.__________________________

predict = ___________________________
accuracy_score(_________________, ________________)"""


'model2 = ________________________________\nmodel2.__________________________\n\npredict = ___________________________\naccuracy_score(_________________, ________________)'

# Deployment

In [238]:
# Command to Open SIMPLE ML Jump 2 (for Example)

# Windows OS
#env_path = "D:\\User\\Desktop\\10botics Data Science\\ML Game\\SimpleMLJump2 Builds\\Windows\\Simple ML Jump 2.exe"

# MacOS
#env_path = 'open -n "/Users/billwaa/Desktop/ML Game/Simple ML Jump 2.app"' 

env_path = "/home/wcyat/dev/kaggle/code/ml-jump/Linux/SimpleMLJump2.x86_64"

In [239]:
# Import Libraries
import socket
import struct
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import time
import subprocess
import platform

# Find Open Socket Ports
def findOpenSockets(size:int):

    sock = []
    port = []

    for i in range(size):
        sock.append(socket.socket())
        sock[i].bind(('localhost', 0))
        port.append(sock[i].getsockname()[1])

    for i in range(len(sock)):
        sock[i].close()


    return port


# Find Open Ports and Launch Game Environment
envNum = 1
openPorts = findOpenSockets(envNum * 2)
gamePort = openPorts[0]
apiPort = openPorts[1]

# Determine OS and Launch 
if platform.system() == 'Darwin':
    env_path += f" --args --apiPort {str(apiPort)} --gamePort {str(gamePort)} --small false"
    process = subprocess.Popen([env_path], shell=True)
else:
    process = subprocess.Popen([env_path, '--apiPort', str(apiPort), '--gamePort', str(gamePort), '--small', 'false'])

# Establish UDP Network Client
localIP     = "localhost"
bufferSize  = 1024
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
UDPServerSocket.bind((localIP, apiPort))

# Set UDP Timeout
UDPServerSocket.settimeout(3)

# Wait for Game Environment to Open
t0 = time.time()

while(time.time() - t0 < 5):
    pass

# Extract Data
def extractData(data):
    dist1 = struct.unpack('f', data[:4])
    speed1 = struct.unpack('f', data[4:8])
    dist2 = struct.unpack('f', data[8:12])
    speed2 = struct.unpack('f', data[12:16])

    onGround = True if (data[16] & (1 << 0)) == 1 else False
    toJump = True if (data[17] & (1 << 0)) == 1 else False
    isDead = True if (data[18] & (1 << 0)) == 1 else False
    scored = True if (data[19] & (1 << 0)) == 1 else False

    dat = [dist1[0], speed1[0], dist2[0], speed2[0], toJump, onGround, isDead, scored] # Swap Order to Accomadate Old Model
    df_temp = pd.DataFrame(dat).transpose()


    return df_temp

# Reset Simulation
UDPServerSocket.sendto(bytes.fromhex('07 00'), (localIP, gamePort))

2

In [240]:
def autoJump():
# Process Loop - Retrieve Data from Simulation, Run Through Model, Output Command
    while True:
        try:

            bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)

            t0 = time.time()

            message = bytesAddressPair[0]
            address = bytesAddressPair[1]
            df_temp = extractData(message)
            X = df_temp.iloc[:, :4]
            X = X.astype(float)

        # Data PreProcessing Function Go Here

        # Model Prediction Function Go Here
            predict = model.predict(X)

            print(predict, end='\r')

        # If Jump Decision is Made, Send to Game
            if predict[0] > 0.5:
            # Jump Command
                UDPServerSocket.sendto(bytes.fromhex('06 01'), (localIP, gamePort))


        except Exception as e:
            print(e)
            break

In [241]:
autoJump()

timed out


In [242]:
# Close UDP Port When Game is Closed
UDPServerSocket.close()